In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras import preprocessing
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
dic_siz=10000 # 사전의 크기(사전에 있는 단어 개수)
sample_siz=512 # 샘플의 크기

In [ ]:
# tensorflow가 제공하는 간소한 버전의 IMDB 읽기
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=dic_siz)

In [ ]:
embed_space_dim=16 # 16차원의 임베딩 공간

In [ ]:
x_train=preprocessing.sequence.pad_sequences(x_train,maxlen=sample_siz)
x_test=preprocessing.sequence.pad_sequences(x_test,maxlen=sample_siz)

In [ ]:
# monitor='val_accuracy' : 멈춤을 판단할 기준으로 검증집합의 정확도를 사용
# patience=5 : 다섯 세대 동안 성능향상이 없을 때 멈추도록 함. 너무 작으면 이후 좋은 성능이 나올 가능성을 놓치고, 너무 크면 시간낭비를 불러올 수 있음.
# restore_best_weights=True : 가장 높은 성능을 발휘했을 때의 가중치를 취한다.
# restore_best_weights=False : 멈춘 순간의 가중치를 취한다.
early=EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True)

In [ ]:
# 신경망 모델의 설계와 학습(LSTM 층 포함)
embed=Sequential()
embed.add(Embedding(input_dim=dic_siz,output_dim=embed_space_dim,input_length=sample_siz))
embed.add(LSTM(units=32))
embed.add(Dense(1,activation='sigmoid'))
embed.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
# callbacks=[early] : 조기멈춤 적용
hist=embed.fit(x_train,y_train,epochs=20,batch_size=64,validation_split=0.2,verbose=2,callbacks=[early])

In [ ]:
# 모델 평가
res=embed.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 학습 곡선
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'], loc='best')
plt.grid()
plt.show()